# HHA550_Diabetes Prediction Dataset

## Healtcare-dataset-stroke-data

#### (Check Modules folders for csv and ipynb for each class)

# DATA
## Diabetes Prediction Dataset


# .CSV Data
### cleaned_diabetic_data.csv

# IMPORTING Everthing

In [185]:
# Commands to install some of the libraries in-case if they are not installed
# Any other library that needs to be installed just use: !pip install <library name>
# !pip install seaborn
# !pip install missingno
# !pip install xgboost
# !pip install catboost
# !pip install regex
# !pip install sklearn
# !pip install pandas
# !pip install numpy
# !pip install imblearn
# !pip install lightgbm

In [186]:
import pandas as pd   # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np   # linear algebra
import matplotlib.pyplot as plt  #graphs and plots
import seaborn as sns   #data visualizations 
import csv # Some extra functionalities for csv  files - reading it as a dictionary
from lightgbm import LGBMClassifier #sklearn is for machine learning and statistical modeling including classification, regression, clustering and dimensionality reduction 

from sklearn.model_selection import train_test_split, cross_validate   #break up dataset into train and test sets

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler


# importing python library for working with missing data
import missingno as msno
# To install missingno use: !pip install missingno
import re    # This library is used to perform regex pattern matching

# import various functions from sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
import xgboost as xgb
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, classification_report, make_scorer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split

Import additional items as needed...
We may not use them all in this course...

In [187]:
from sklearn.model_selection import KFold,cross_val_score, RepeatedStratifiedKFold,StratifiedKFold
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import OneHotEncoder,StandardScaler,PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer,SimpleImputer
from sklearn.compose import make_column_transformer
from imblearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.impute import SimpleImputer
from sklearn.dummy import DummyClassifier
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score,\
                            precision_score, recall_score, roc_auc_score,\
                            ConfusionMatrixDisplay, classification_report, RocCurveDisplay, f1_score
from sklearn.linear_model import LinearRegression
import plotly 
import plotly.express as px
import plotly.graph_objs as go
import plotly.offline as py
from plotly.offline import iplot
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

import warnings
warnings.filterwarnings("ignore")

* If from imblearn.oversampling import SMOTE does not load use
    `conda install -c conda-forge imbalanced-learn`
* Then rerun
    `from imblearn.over_sampling import SMOTE`

# Exploratory Data Analysis (EDA)

## Start with Loading the CSV Data

In [188]:
dm = pd.read_csv('/Users/lozo/Developer/AHI_Github/HHA550_Analysis/data/cleaned_diabetic_data1.csv')
dm_analysis = dm[['readmitted', 'age', 'weight', 'diabetesMed']]

# Breaking the data up into Train & Test

In [189]:
train_df, valid_df, test_df = np.split(dm_analysis.sample(frac=1, random_state=42), 
                                        [int(.7*len(dm_analysis)), int(0.85*len(dm_analysis))])
train_df = train_df.reset_index(drop = True)
valid_df = valid_df.reset_index(drop = True)
test_df = test_df.reset_index(drop = True)

In [190]:
dm_analysis.readmitted.value_counts()

readmitted
0    66221
1    35545
Name: count, dtype: int64

In [191]:
train_df.readmitted.value_counts()

readmitted
0    46318
1    24918
Name: count, dtype: int64

In [192]:
valid_df.readmitted.value_counts()

readmitted
0    10008
1     5257
Name: count, dtype: int64

In [193]:
test_df.readmitted.value_counts()

readmitted
0    9895
1    5370
Name: count, dtype: int64

# Treating the Imbalance in the Data

Imbalance in the data means that one of the classes in the data is too less as compared to the others. Typically, it is better to balance the data in some way to give the positives more weight. There are 3 strategies that are typically utilized:

* Sub-sample the more dominant class: use a random subset of the negatives
* Over-sample the imbalanced class: use the same positive samples multiple times
* Create synthetic positive data

Usually, you will want to use the latter two methods if you only have a handful of positive cases. Since we have a few thousand positive cases, let's use the sub-sample approach. Here, we will create a balanced training data set that has 50% positive and 50% negative. You can also play with this ratio to see if you can get an improvement.

In [194]:
def calc_prevalence(y_actual):
    
    '''
    This function is to understand the ratio/distribution of the classes that we are going to predict for.
    
    Params:
    1. y_actual: The target feature
    
    Return:
    1. (sum(y_actual)/len(y_actual)): The ratio of the postive class in the comlpete data.
    '''
    
    return (sum(y_actual)/len(y_actual))

In [195]:
# split the training data into positive and negative
rows_pos = train_df.readmitted == 1
df_train_pos = train_df.loc[rows_pos]
df_train_neg = train_df.loc[~rows_pos]

# merge the balanced data
dm_df_balanced = pd.concat([df_train_pos, df_train_neg.sample(n = len(df_train_pos), random_state = 111)],axis = 0)

# shuffle the order of training samples 
dm_df_balanced = dm_df_balanced.sample(n = len(dm_df_balanced), random_state = 42).reset_index(drop = True)

print('Train balanced prevalence(n = %d):%.3f'%(len(dm_df_balanced), \
                                                calc_prevalence(dm_df_balanced.readmitted.values)))

Train balanced prevalence(n = 49836):0.500


In [196]:
dm_df_balanced.readmitted.value_counts()

readmitted
1    24918
0    24918
Name: count, dtype: int64

In [197]:
X_train = dm_df_balanced.drop('readmitted',axis=1)

y_train = dm_df_balanced['readmitted']

X_valid = valid_df.drop('readmitted',axis=1)

y_valid = valid_df['readmitted']

X_test = test_df.drop('readmitted',axis=1)

y_test = test_df['readmitted']

In [198]:
scaler=StandardScaler()
X_train[['age', 'weight', 'diabetesMed']] = pd.DataFrame(scaler.fit_transform(X_train[['age', 'weight', 'diabetesMed']]),columns=['age', 'weight', 'diabetesMed'])
X_valid[['age', 'weight', 'diabetesMed']] = pd.DataFrame(scaler.transform(X_valid[['age', 'weight', 'diabetesMed']]),columns=['age', 'weight', 'diabetesMed'])
X_test[['age', 'weight', 'diabetesMed']] = pd.DataFrame(scaler.transform(X_test[['age', 'weight', 'diabetesMed']]),columns=['age', 'weight', 'diabetesMed'])

# Creating and Understanding Models

In [199]:
def calc_specificity(y_actual, y_pred, thresh):
    # calculates specificity
    return sum((y_pred < thresh) & (y_actual == 0)) /sum(y_actual ==0)

def print_report(y_actual, y_pred, thresh = 0.5):
    
    '''
    This function calculates all the metrics to asses the machine learning models.
    
    Params:
    1. y_actual: The actual values for the target variable.
    2. y_pred: The predicted values for the target variable.
    3. thresh: The threshold for the probability to be considered as a positive class. Default value 0.5
    
    Return:
    1. AUC
    2. Accuracy
    3. Recall
    4. Precision
    5. Specificity
    '''
    
    auc = roc_auc_score(y_actual, y_pred)
    accuracy = accuracy_score(y_actual, (y_pred > thresh))
    recall = recall_score(y_actual, (y_pred > thresh))
    precision = precision_score(y_actual, (y_pred > thresh))
    specificity = calc_specificity(y_actual, y_pred, thresh)
    print('AUC:%.3f'%auc)
    print('accuracy:%.3f'%accuracy)
    print('recall:%.3f'%recall)
    print('precision:%.3f'%precision)
    print('specificity:%.3f'%specificity)
    print('prevalence:%.3f'%calc_prevalence(y_actual))
    print(' ')
    return auc, accuracy, recall, precision, specificity

## Linear Regression

In [200]:
lnr = LinearRegression()
lnr.fit(X_train, y_train)

y_valid_preds = lnr.predict(X_valid)

In [201]:
y_valid_preds

array([0.52593862, 0.45743958, 0.51497146, 0.50503466, 0.52593862,
       0.51549342, 0.49448881, 0.51549342, 0.46680392, 0.47671298,
       0.52541756, 0.46784433, 0.45692097, 0.51549342, 0.51549342,
       0.47723431, 0.52541756, 0.51549342, 0.51601534, 0.45795828,
       0.51444948, 0.44449036, 0.52385405, 0.50555704, 0.52437527,
       0.52489644, 0.45588404, 0.49613867, 0.50555704, 0.45795828,
       0.51497146, 0.50503466, 0.50660176, 0.4761917 , 0.46732409,
       0.50607941, 0.46732409, 0.45795828, 0.50660176, 0.46732409,
       0.50555704, 0.45743958, 0.50346746, 0.50555704, 0.50607941,
       0.50294504, 0.51549342, 0.51497146, 0.46732409, 0.51601534,
       0.52489644, 0.55349003, 0.51549342, 0.51392746, 0.50607941,
       0.51601534, 0.51444948, 0.51549342, 0.47462818, 0.45743958,
       0.50555704, 0.52541756, 0.51653722, 0.51549342, 0.48463501,
       0.54467977, 0.43726405, 0.46628382, 0.50555704, 0.46732409,
       0.51497146, 0.47775569, 0.50503466, 0.51392746, 0.49457

## Logistic Regression

In [202]:
lr=LogisticRegression(random_state = 42, solver = 'newton-cg', max_iter = 200)
lr.fit(X_train, y_train)

y_valid_preds = lr.predict_proba(X_valid)[:,1]

print('Metrics for Validation data:')

lr_valid_auc, lr_valid_accuracy, lr_valid_recall, \
    lr_valid_precision, lr_valid_specificity = print_report(y_valid,y_valid_preds, 0.5)

Metrics for Validation data:
AUC:0.539
accuracy:0.474
recall:0.693
precision:0.362
specificity:0.359
prevalence:0.344
 


## Explaining Results

In [203]:
from IPython import display
# display.Image("C:/Users/kaden/Desktop/HHA550_Stroke_Data/Explaining_Results.png")

Lets look at some other models to see if we get better results.

## KNN Model

In [204]:
knn = KNeighborsClassifier(n_neighbors = 100)
knn.fit(X_train, y_train)

knn_preds = knn.predict_proba(X_valid)[:,1]

lr_valid_auc, lr_valid_accuracy, lr_valid_recall, \
    lr_valid_precision, lr_valid_specificity = print_report(y_valid,knn_preds, 0.5)

AUC:0.523
accuracy:0.501
recall:0.563
precision:0.357
specificity:0.461
prevalence:0.344
 


## Stochastic Gradient Descent Model

In [205]:
sgdc=SGDClassifier(loss = 'log',alpha = 0.1,random_state = 42)
sgdc.fit(X_train, y_train)

sgd_preds = sgdc.predict_proba(X_valid)[:,1]

print('Stochastic Gradient Descent')
print('Validation:')
sgdc_valid_auc, sgdc_valid_accuracy, sgdc_valid_recall, \
                sgdc_valid_precision, sgdc_valid_specificity = print_report(y_valid,sgd_preds, 0.5)

Stochastic Gradient Descent
Validation:
AUC:0.539
accuracy:0.474
recall:0.693
precision:0.362
specificity:0.359
prevalence:0.344
 


## Decision Tree

In [206]:
dc_clf = DecisionTreeClassifier(random_state=42, max_depth = 10)
dc_clf.fit(X_train, y_train)

dc_preds_proba = dc_clf.predict_proba(X_valid)[:,1]
dc_preds = dc_clf.predict(X_valid)

lr_valid_auc, lr_valid_accuracy, lr_valid_recall, \
    lr_valid_precision, lr_valid_specificity = print_report(y_valid,dc_preds_proba, 0.5)

AUC:0.540
accuracy:0.480
recall:0.671
precision:0.363
specificity:0.379
prevalence:0.344
 


## Random Forest

In [207]:
rf_clf = RandomForestClassifier(random_state=111, max_depth = 6)

rf_clf.fit(X_train, y_train)

rf_preds = rf_clf.predict(X_valid)
rf_preds_proba = rf_clf.predict_proba(X_valid)[:, 1]

lr_valid_auc, lr_valid_accuracy, lr_valid_recall, \
    lr_valid_precision, lr_valid_specificity = print_report(y_valid,rf_preds_proba, 0.5)

AUC:0.541
accuracy:0.481
recall:0.673
precision:0.363
specificity:0.379
prevalence:0.344
 


## Linear SVC

In [208]:
lsvc_clf = LinearSVC(random_state=111)
lsvc_clf.fit(X_train, y_train)

lsvc_preds = lsvc_clf.decision_function(X_valid)

lr_valid_auc, lr_valid_accuracy, lr_valid_recall, \
    lr_valid_precision, lr_valid_specificity = print_report(y_valid,lsvc_preds, 0.5)

AUC:0.539
accuracy:0.656
recall:0.000
precision:0.000
specificity:1.000
prevalence:0.344
 


## Gradient Boosting Model

In [209]:
gb_clf = GradientBoostingClassifier(n_estimators = 100, criterion='friedman_mse', learning_rate = 1.0, max_depth = 3,\
                                    random_state = 111)

gb_clf.fit(X_train, y_train)
gb_preds = gb_clf.predict(X_valid)
gb_preds_proba = gb_clf.predict_proba(X_valid)[:, 1]

lr_valid_auc, lr_valid_accuracy, lr_valid_recall, \
    lr_valid_precision, lr_valid_specificity = print_report(y_valid,gb_preds_proba, 0.5)

AUC:0.539
accuracy:0.481
recall:0.672
precision:0.363
specificity:0.380
prevalence:0.344
 


## XGB Model

In [210]:
xgb_clf = xgb.XGBClassifier(max_depth=3, learning_rate = 1.0, use_label_encoder = False,\
                            eval_metric = 'logloss')
xgb_clf.fit(X_train, y_train)

xgb_preds = xgb_clf.predict(X_valid)
xgb_preds_proba = xgb_clf.predict_proba(X_valid)[:, 1]

lr_valid_auc, lr_valid_accuracy, lr_valid_recall, \
    lr_valid_precision, lr_valid_specificity = print_report(y_valid,xgb_preds_proba, 0.5)

AUC:0.539
accuracy:0.480
recall:0.671
precision:0.363
specificity:0.380
prevalence:0.344
 


## Catboost Model

In [211]:
catb=CatBoostClassifier(iterations=200, depth=3, learning_rate=1.0, random_state = 111)
catb.fit(X_train, y_train)
catb_preds = catb.predict_proba(X_valid)[:, 1]

lr_valid_auc, lr_valid_accuracy, lr_valid_recall, \
    lr_valid_precision, lr_valid_specificity = print_report(y_valid,catb_preds, 0.5)

0:	learn: 0.6903860	total: 6.99ms	remaining: 1.39s
1:	learn: 0.6901112	total: 13.6ms	remaining: 1.34s
2:	learn: 0.6899598	total: 18.3ms	remaining: 1.2s
3:	learn: 0.6897998	total: 22.8ms	remaining: 1.12s
4:	learn: 0.6897434	total: 27.2ms	remaining: 1.06s
5:	learn: 0.6896810	total: 32ms	remaining: 1.03s
6:	learn: 0.6896748	total: 36ms	remaining: 993ms
7:	learn: 0.6896659	total: 40.8ms	remaining: 978ms
8:	learn: 0.6896659	total: 44.4ms	remaining: 943ms
9:	learn: 0.6896546	total: 49.7ms	remaining: 944ms
10:	learn: 0.6896373	total: 54.7ms	remaining: 939ms
11:	learn: 0.6896364	total: 59.2ms	remaining: 927ms
12:	learn: 0.6896325	total: 63.3ms	remaining: 911ms
13:	learn: 0.6896174	total: 67.1ms	remaining: 892ms
14:	learn: 0.6896155	total: 71.1ms	remaining: 876ms
15:	learn: 0.6895637	total: 76.1ms	remaining: 875ms
16:	learn: 0.6895622	total: 79.7ms	remaining: 858ms
17:	learn: 0.6895588	total: 83.8ms	remaining: 848ms
18:	learn: 0.6895572	total: 87.5ms	remaining: 833ms
19:	learn: 0.6895313	total:

# Hyper Parameter Tuning

* From the above models we will choose two models for demonstration i.e. Random Forest, Decision Trees for hyper-parameter tuning.
* Generally you can pick up the top three models based on the 'AUC', 'Recall' or 'F1 score' score and tune them.

There are many techniques for hyper-parameter tuning:

* Random Search
* Grid Search
* Halving Grid Search(added recently in sklearn)

Special Note:
* It will take significant time to run Hyper Parameter Tuning 
* Timing will depend on available resources of server

In [212]:
recall_scoring = make_scorer(recall_score)

## Decision Tree - Hyper Parameter Tuning

In [213]:
dc_grid = {'max_features':['auto','sqrt'], # maximum number of features to use at each split
           'max_depth':range(1,11,1), # maximum depth of the tree
           'min_samples_split':range(2,10,2), # minimum number of samples to split a node
           'criterion':['gini','entropy']} # criterion for evaluating a split

dc_random = RandomizedSearchCV(estimator = dc_clf, param_distributions = dc_grid, 
                               n_iter = 20, cv = 2, scoring=recall_scoring,
                               verbose = 1, random_state = 111)

dc_random.fit(X_train, y_train)

dc_random.best_params_

dc_hp_preds = dc_random.best_estimator_.predict(X_valid)
dc_hp_preds_proba = dc_random.best_estimator_.predict_proba(X_valid)[:,1]
roc_auc_score(y_valid, dc_hp_preds_proba)

Fitting 2 folds for each of 20 candidates, totalling 40 fits


0.5289312301347813

In [214]:
recall_score(y_valid, dc_hp_preds)

0.7863800646756706

## Random Forest - Hyper Parameter Tuning

In [215]:
rf_grid = {'n_estimators':range(200,1000,200), # number of trees
           'max_features':['auto','sqrt'], # maximum number of features to use at each split
           'max_depth':range(1,11,1), # maximum depth of the tree
           'min_samples_split':range(2,10,2), # minimum number of samples to split a node
           'criterion':['gini','entropy']} # criterion for evaluating a split

rf_random = RandomizedSearchCV(estimator = rf_clf, param_distributions = rf_grid, 
                               n_iter = 20, cv = 2, scoring=recall_scoring,
                               verbose = 1, random_state = 111)

rf_random.fit(X_train, y_train)

rf_random.best_params_

rf_hp_preds = rf_random.best_estimator_.predict(X_valid)
rf_hp_preds_proba = rf_random.best_estimator_.predict_proba(X_valid)[:,1]
roc_auc_score(y_valid, rf_hp_preds_proba)

Fitting 2 folds for each of 20 candidates, totalling 40 fits


0.5391647496155634

In [216]:
recall_score(y_valid, rf_hp_preds)

0.7859996195548792

## XGBoost - Hyper Parameter Tuning

In [217]:
# xgb_grid = params = {
#         'min_child_weight': [1, 5, 8, 10],
#         'gamma': [0.5, 1, 1.5, 2, 5],
#         'subsample': [0.6, 0.8, 1.0],
#         'colsample_bytree': [0.6, 0.8, 0.9, 1.0],
#         'max_depth': [3, 4, 5]
#         } # criterion for evaluating a split

# xgb_random = GridSearchCV(estimator = xgb_clf, param_grid = xgb_grid, 
#                                cv = 2, scoring = recall_scoring,
#                                verbose = 1)

# xgb_random.fit(X_train, y_train)

# xgb_random.best_params_

# xgb_hp_preds = xgb_random.best_estimator_.predict(X_valid)
# xgb_hp_preds_proba = xgb_random.best_estimator_.predict_proba(X_valid)[:,1]
# roc_auc_score(y_valid, xgb_hp_preds_proba)

In [218]:
# recall_score(y_valid, xgb_hp_preds)